In [135]:
#import the dependencies

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [136]:
credit_card_data=pd.read_csv('creditcard.csv')

# dataset information
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

Checking the number of missing values in each column

In [137]:
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

#Distribution of legit transaction and fraudulent transaction

In [138]:
credit_card_data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

From the above data we can see that the dataset is highly unbalanced. 
Let 0 represent normal transaction
and 1 represent fraudulent transaction

Seperating the data for analysis

In [139]:
legit = credit_card_data[credit_card_data.Class==0]
fraud = credit_card_data[credit_card_data.Class==1]
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


Obtaining statistical measures of the data

In [140]:
print('Statistical data of legit transaction')
legit.Amount.describe()

Statistical data of legit transaction


count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [141]:
print('Statistical data of fraud transaction')
fraud.Amount.describe()

Statistical data of fraud transaction


count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

For all the fraud transaction the mean transacted amount is higher than the mean of normal transaction

compare the values for both transactions

In [142]:
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Under-Sampling
Build a sample data set containing similar distribution of normal transactions and fraudulent transactions

In [143]:
legit_sample = legit.sample(n=492)

Concatenate two dataframes

In [144]:
new_dataset= pd.concat([legit_sample,fraud],axis=0)
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
235739,148521.0,-1.571957,-0.631464,0.497676,-2.231268,1.679677,-3.002716,1.016240,-0.799071,-1.589042,...,0.417873,1.316516,-0.982098,0.970602,1.475978,0.166483,-0.220653,-0.295696,20.40,0
260271,159474.0,0.061528,0.836143,0.007860,-0.740298,0.802166,-0.729298,1.030315,-0.191489,-0.074184,...,-0.294060,-0.659034,-0.002142,-0.648451,-0.375940,0.179255,0.246454,0.092568,8.95,0
222611,143050.0,2.064189,0.020080,-1.106143,0.373733,0.033658,-1.098335,0.272049,-0.397194,0.363556,...,-0.281774,-0.627409,0.302818,-0.076215,-0.238584,0.200414,-0.063752,-0.057925,6.67,0
250852,155102.0,1.980346,-0.028747,-1.523419,0.630586,0.211017,-0.733366,0.034546,-0.243050,0.669778,...,-0.120439,-0.231069,0.074963,-0.718439,-0.109068,0.253842,-0.028075,-0.023590,49.95,0
109560,71436.0,1.164627,0.034294,-0.218267,-0.135819,0.168396,-0.363380,0.316878,-0.095411,-0.475493,...,-0.291186,-0.942483,0.080923,-0.253320,0.156321,0.729765,-0.099052,-0.008221,49.94,0


Checking the number of fraud and legit transaction

In [145]:
new_dataset['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [146]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,97832.280488,0.051940,-0.004750,0.020522,-0.067773,0.013916,-0.046912,-0.030331,0.059461,0.013568,...,0.003433,0.043307,0.048432,0.006329,-0.032344,-0.028819,0.002364,-0.009678,-0.014650,74.518598
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Spliting the dataset into Features and Targets

In [147]:
X=new_dataset.drop(columns='Class',axis=1)
Y=new_dataset['Class']

In [148]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
235739  148521.0 -1.571957 -0.631464  0.497676 -2.231268  1.679677 -3.002716   
260271  159474.0  0.061528  0.836143  0.007860 -0.740298  0.802166 -0.729298   
222611  143050.0  2.064189  0.020080 -1.106143  0.373733  0.033658 -1.098335   
250852  155102.0  1.980346 -0.028747 -1.523419  0.630586  0.211017 -0.733366   
109560   71436.0  1.164627  0.034294 -0.218267 -0.135819  0.168396 -0.363380   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [149]:
print(Y)

235739    0
260271    0
222611    0
250852    0
109560    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


Split the data to train data and test data

In [150]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

Training the model - Logistic regression

In [151]:
model = LogisticRegression()

#Training the logistic regression model with training data

model.fit(X_train,Y_train)

LogisticRegression()

Model Evaluation - Accuracy score

In [152]:
#accuracy on training data
X_train_prediction=model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9275730622617535


In [153]:
#accuracy on test data
X_test_prediction=model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)
print('Accuracy on test data : ', test_data_accuracy)

Accuracy on test data :  0.9238578680203046
